# Real Time Video Emotion Recognition

This notebook will go through training a simple model to classify the emotions on someone's face.

This model is loaded by a script that will monitor these emotions in real time.

### 1) General Set up

In [ ]:
# mount drive to colab.

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#Imports 

import sys, os
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,AveragePooling2D
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils import np_utils

### 2) Load and Transform Data 

In [ ]:
#load data and look at the df.

df=pd.read_csv('gdrive/My Drive/fer2013.csv')

print(df.head())

   emotion                                             pixels     Usage
0        0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...  Training
1        0  151 150 147 155 148 133 111 140 170 174 182 15...  Training
2        2  231 212 156 164 174 138 161 173 182 200 106 38...  Training
3        4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...  Training
4        6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...  Training


In [ ]:
#build the training and test sets 
X_train,train_y,X_test,test_y=[],[],[],[]

for index, row in df.iterrows():
    val=row['pixels'].split(" ")
    try:
        if 'Training' in row['Usage']:
           X_train.append(np.array(val,'float32'))
           train_y.append(row['emotion'])
        elif 'PublicTest' in row['Usage']:
           X_test.append(np.array(val,'float32'))
           test_y.append(row['emotion'])
    except:
        print(f"error occured at index :{index} and row:{row}")


In [ ]:
#set hyperparameters and preprocess data.

num_features = 64
num_labels = 7
batch_size = 64
epochs = 200
width, height = 48, 48

X_train = np.array(X_train,'float32')
train_y = np.array(train_y,'float32')
X_test = np.array(X_test,'float32')
test_y = np.array(test_y,'float32')

train_y=np_utils.to_categorical(train_y, num_classes=num_labels)
test_y=np_utils.to_categorical(test_y, num_classes=num_labels)

In [ ]:
#normalizing data between 0 and 1
X_train -= np.mean(X_train, axis=0)
X_train /= np.std(X_train, axis=0)

X_test -= np.mean(X_test, axis=0)
X_test /= np.std(X_test, axis=0)

X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)
X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)

print(f"shape:{X_train.shape}")

shape:(28709, 48, 48, 1)


### 3) Build and train CNN

In [ ]:
##designing the cnn
#1st convolution layer
model = Sequential()

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(X_train.shape[1:])))
model.add(Conv2D(64,kernel_size= (3, 3), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))

#2nd convolution layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))

#3rd convolution layer
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(Conv2D(256, (3, 3), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))

model.add(Flatten())

#fully connected neural networks
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(num_labels, activation='softmax'))

model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_55 (Conv2D)           (None, 46, 46, 64)        640       
_________________________________________________________________
conv2d_56 (Conv2D)           (None, 44, 44, 64)        36928     
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 22, 22, 64)        0         
_________________________________________________________________
dropout_28 (Dropout)         (None, 22, 22, 64)        0         
_________________________________________________________________
conv2d_57 (Conv2D)           (None, 20, 20, 128)       73856     
_________________________________________________________________
conv2d_58 (Conv2D)           (None, 18, 18, 128)       147584    
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 9, 9, 128)        

In [ ]:
#Compliling the model
model.compile(loss=categorical_crossentropy,
              optimizer=Adam(),
              metrics=['accuracy'])


In [ ]:
#Training the model
model.fit(X_train, train_y,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, test_y),
          shuffle=True)

Epoch 1/200
449/449 [==============================] - 7s 15ms/step - loss: 1.7546 - accuracy: 0.2758 - val_loss: 1.5643 - val_accuracy: 0.3934
Epoch 2/200
449/449 [==============================] - 6s 14ms/step - loss: 1.5019 - accuracy: 0.4091 - val_loss: 1.4380 - val_accuracy: 0.4586
Epoch 3/200
449/449 [==============================] - 6s 14ms/step - loss: 1.3759 - accuracy: 0.4699 - val_loss: 1.2973 - val_accuracy: 0.4960
Epoch 4/200
449/449 [==============================] - 6s 14ms/step - loss: 1.3080 - accuracy: 0.4981 - val_loss: 1.2592 - val_accuracy: 0.5127
Epoch 5/200
449/449 [==============================] - 6s 14ms/step - loss: 1.2579 - accuracy: 0.5183 - val_loss: 1.2332 - val_accuracy: 0.5297
Epoch 6/200
449/449 [==============================] - 6s 14ms/step - loss: 1.2157 - accuracy: 0.5337 - val_loss: 1.1837 - val_accuracy: 0.5419
Epoch 7/200
449/449 [==============================] - 6s 14ms/step - loss: 1.1807 - accuracy: 0.5486 - val_loss: 1.1788 - val_accuracy:

### 4) Export trained model

In [ ]:
#change directory into drive
%cd /content/gdrive/My Drive/Emotion

/content/gdrive/My Drive/Emotion


In [ ]:
#Saving the  model to  use it later on
fer_json2 = model.to_json()
with open("fer2.json", "w") as json_file:
    json_file.write(fer_json2)
model.save_weights("fer2.h5")